In [ ]:
%pip install dask
%pip install dask[dataframe]
%pip install dask[complete]
%pip install dask[distributed]
%pip install cloudpickle
%pip install graphviz

In [ ]:
%pip install transformers
%pip install pytorch_transformers

In [ ]:
# define the imports
try:
    import os
    import json
    import math
    import dask
    from dask.distributed import Client
    import dask.dataframe as dd
    from dask.diagnostics import ProgressBar
    import dask.multiprocessing
    import numpy as np
    import torch
    from transformers import BertTokenizer
    from transformers import BertModel
    from tqdm.notebook import tqdm
    tqdm.pandas()
except Exception as ex:
    print('Some modules are missing : {}'.format(ex))

In [ ]:
os.listdir()

In [ ]:
size = os.path.getsize('../data/allData-19-08-27.ta') / math.pow(1024, 3)
print('Size in GB: {}'.format(size))

In [ ]:
client = Client(n_workers = 5, threads_per_worker = 1, processes = False, memory_limit = '2GB')
client

In [ ]:
dtypes = {
    'id': str,
    'news': str    
}

In [ ]:
ddf = dd.read_csv('../data/news_with_header.csv',
                 names=['id', 'news'], dtype=dtypes)
ddf

In [ ]:
ddf.shape

In [ ]:
ddf.columns

> Vectorization

In [ ]:
## Load pretrained model/tokenizer
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-multilingual-uncased', do_lower_case=True)
model = BertModel.from_pretrained(
    'bert-base-multilingual-uncased', output_hidden_states=True)

In [ ]:
# Put the model in "evaluation" mode,meaning feed-forward operation.
model.eval()

In [ ]:
def get_vectors(row):
  print('e')
  encoding = tokenizer.encode(row['news'], add_special_tokens=True, max_length=512, truncation=True, padding="max_length")
  token_text = tokenizer.convert_ids_to_tokens(encoding)
  indexed_tokens = tokenizer.convert_tokens_to_ids(token_text)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])

  # Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
  with torch.no_grad():
    outputs = model(tokens_tensor)

    # Evaluating the model will return a different number of objects based on how it's  configured in the `from_pretrained` call earlier.
    # In this case, becase we set `output_hidden_states = True`, the third item will be the hidden states from all layers.
    # See the documentation for more details:https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

    # initial embeddings can be taken from 0th layer of hidden states
    word_embed = hidden_states[0]
    
    output_path = '../data/vectors/{id}.npy'.format(id = row['id'])
    np.save(output_path, word_embed)
    # return word_embed

In [ ]:
parallel_task = []
parallel_task.append(dask.delayed(get_vectors))


In [ ]:
get_vectors = dask.delayed(get_vectors)

In [ ]:
p = ddf.map_partitions(get_vectors)

In [ ]:
# p.visualize()

In [ ]:
p.compute()

In [ ]:
p[0][0][1].compute()